In [1]:
# takes the path of PDB file using os module to join the directory in a platform-independent way
import os
DATA = os.path.join('/home', 'uni', 'capstone', 'EROS_Ub/')
print(DATA)

/home/uni/capstone/EROS_Ub/


In [2]:
#creates a universe object from the PDB file alone

import MDAnalysis as mda
mda_universe = mda.Universe(os.path.join(DATA, "2k39.pdb")) 
# why is this object creation taking only topology file and no trajectory file
print(mda_universe)

<Universe with 1231 atoms>


In [ ]:
# This function does many things:
# 1. It takes a mdanalysis universe object
# 2. It takes an optional argument selection string to select atoms to visualize
# 3. It also takes an optional argument style to set the visualization style; not imp at the moment

# 5. Cleans up previously stored PDBs storing frames
# 6. It creates object blueprints for atoms and molecules in the PDB files to be read;
#   there are two use cases for the atom and molecule objects: to print the object, taken care of by __str__ method
#   and to write the object to a PDB file, taken care of by __str__ method;
#   the __init__ method takes care of reading the PDB file and creating the object

# 7. write frames to files
# 8. list of molecule objects is created and a string of all molecules called models is created

def visualize_mda_universe(u, \
    sel_string='not ((resname WAT) or (resname HOH))', 
    style={"cartoon": {'color': 'spectrum'}}):
  
  """
  Inputs: 
  u : mdanalysis universe
  sel_string : mdanalysis selection string for visible atoms
  style : py3Dmol style
  """


  # The number of frames in the simulation
  number_frames_analysis = len(u.trajectory)
  if number_frames_analysis > 10:
    stride_animation = number_frames_analysis/1 #100 KC
  else:
    stride_animation = 1

  # Deleting previously stored frames as PDBs and removing warnings
  import warnings
  warnings.filterwarnings('ignore')
  !rm [0-9]?.pdb 2> /dev/null
  

  # TODO: do i need to assert that the line has no more than 80 characters?
    # Helper classes to read and get PDB fields
  class Atom(dict):
    def __init__(self, line):
      self["type"] = line[0:6].strip()
      self["idx"] = line[6:11].strip()
      self["name"] = line[12:16].strip()      # TODO : what is this
      self["resname"] = line[17:20].strip()   # TODO: there is something between residue name and residue id what is it, in the file
      self["resid"] = int(int(line[22:26]))
      #print(type(self["resid"]))
      #print(self["resid"])  
      self["x"] = float(line[30:38])
      self["y"] = float(line[38:46])
      self["z"] = float(line[46:54])          
      # there are two thing after x y and z coordinates, and before symbol of atom; 
      # The second one is B-factor or temperature factor (as explained in one of the papers in the literature)
      # The first one is a quantity describing the amount of time spent 
      # in that position described by x y and z; it's called occupancy 
      # # Occupancy comes in handy to note if the molecule is flexible in that region
      self["sym"] = line[76:78].strip()

    def __str__(self):
      line = list(" " * 80)                   # initializes a line with 80 characters
      line[0:6] = self["type"].ljust(6)       
      line[6:11] = self["idx"].ljust(5)
      line[12:16] = self["name"].ljust(4)
      line[17:20] = self["resname"].ljust(3)
      line[22:26] = str(self["resid"]).ljust(4)
      line[30:38] = str(self["x"]).rjust(8)
      line[38:46] = str(self["y"]).rjust(8)
      line[46:54] = str(self["z"]).rjust(8)
      line[76:78] = self["sym"].rjust(2)
      return "".join(line) + "\n"             # returns the line after updating the characters to reflect atom attributes
          
  class Molecule(list):
    def __init__(self, file):                 # collects all lines labelled ATOM
      for line in file:
        if "ATOM  " in line or "HETATM" in line:      # TODO: what is HETATM; heterogeneous atom is an atom in a small molecule
          self.append(Atom(line))
              
      def __str__(self):
        outstr = ""
        for at in self:
          outstr += str(at)
        return outstr

  # Write out frames for animation
  protein = u.select_atoms(sel_string) 
  i = 0
  for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:  # TODO: why unused variable ts
      if i > -1:
          with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:  
              W.write(protein)
      i = i + 1

  # TODO: why write to files at all if all you are going to do with the files is to  put them all in the string models later


  # Load frames as molecules (py3Dmol let us visualize a single "molecule" per frame)
  molecules = []
  for i in range(int(len(u.trajectory)/int(stride_animation))):
      with open('' + str(i) + '.pdb') as ifile:
          molecules.append(Molecule(ifile))
  print(molecules)
  models = ""
  for i in range(len(molecules)):
    models += "MODEL " + str(i) + "\n"
    for j,mol in enumerate(molecules[i]):  # TODO: shouldn't the variable here be atom instead of mol?
      # i think it should be atom;
      # TODO: figure out a way to get rid of the useless j variable
      models += str(mol)
    models += "ENDMDL\n"


  # Animation
  import py3Dmol
  view = py3Dmol.view(width=800, height=600)
  view.addModelsAsFrames(models)
  print(models)
  for i, at in enumerate(molecules[0]):
      view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", style))

  view.zoomTo()
  view.animate({'loop': "forward", 'reps': 0}) # TODO: explain this line
  return view

In [6]:
#creating a view object of py#Dmol module to view the animation

view = visualize_mda_universe(mda_universe)
view.show()

[[{'type': 'ATOM', 'idx': '1', 'name': 'N', 'resname': 'MET', 'resid': 1, 'x': 13.434, 'y': 30.709, 'z': 16.715, 'sym': 'N'}, {'type': 'ATOM', 'idx': '2', 'name': 'CA', 'resname': 'MET', 'resid': 1, 'x': 13.659, 'y': 30.3, 'z': 18.11, 'sym': 'C'}, {'type': 'ATOM', 'idx': '3', 'name': 'C', 'resname': 'MET', 'resid': 1, 'x': 14.899, 'y': 31.03, 'z': 18.626, 'sym': 'C'}, {'type': 'ATOM', 'idx': '4', 'name': 'O', 'resname': 'MET', 'resid': 1, 'x': 15.953, 'y': 30.604, 'z': 18.157, 'sym': 'O'}, {'type': 'ATOM', 'idx': '5', 'name': 'CB', 'resname': 'MET', 'resid': 1, 'x': 13.679, 'y': 28.767, 'z': 18.194, 'sym': 'C'}, {'type': 'ATOM', 'idx': '6', 'name': 'CG', 'resname': 'MET', 'resid': 1, 'x': 13.568, 'y': 28.117, 'z': 19.57, 'sym': 'C'}, {'type': 'ATOM', 'idx': '7', 'name': 'SD', 'resname': 'MET', 'resid': 1, 'x': 13.587, 'y': 26.307, 'z': 19.538, 'sym': 'S'}, {'type': 'ATOM', 'idx': '8', 'name': 'CE', 'resname': 'MET', 'resid': 1, 'x': 15.369, 'y': 25.978, 'z': 19.566, 'sym': 'C'}, {'type

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [57]:
# TODO: what does alignment do?

from MDAnalysis.analysis import align

# Set trajectory pointer to first frame as reference
mda_universe.trajectory[0]

# in_memory=True is needed to actually modify the loaded trajectory
# if the trajectory is too big for memory, one can write out the aligned trajectory and reload it into a new universe
alignment = align.AlignTraj(
    mobile=mda_universe, reference=mda_universe, select="protein", in_memory=True
)
alignment.run()

In [ ]:
view = visualize_mda_universe(mda_universe)
view.show()

In [59]:
# rmsd for atomgroups meaning deviation from which structure;
# the reference structure is drawn from the parameter universe
# the function also takes the atomgroup
# what is the second atomgroup ie what is selection2

import numpy as np
import pandas as pd
from MDAnalysis.analysis import rms

def rmsd_for_atomgroups(universe, selection1, selection2=None):
    """Calulate the RMSD for selected atom groups.

    Parameters
    ----------
    universe: MDAnalysis.core.universe.Universe
        MDAnalysis universe.
    selection1: str
        Selection string for main atom group, also used during alignment.
    selection2: list of str, optional
        Selection strings for additional atom groups.

    Returns
    -------
    rmsd_df: pandas.core.frame.DataFrame
        DataFrame containing RMSD of the selected atom groups over time.
    """
    print(len(universe.trajectory))
    universe.trajectory[0]   # what is this line doing? seems to seek to the 0th frame such that universe is in that frame; 
    # why would this happen in this function that returns rmsd of given atomgroups
    ref = universe
    rmsd_analysis = rms.RMSD(universe, ref, select=selection1, groupselections=selection2)
    rmsd_analysis.run()     # why is there a run method here? it wouldn't surprise me if it not for the method in the previous line
    columns = [selection1, *selection2] if selection2 else [selection1]
    rmsd_df = pd.DataFrame(np.round(rmsd_analysis.rmsd[:, 2:], 2), columns=columns)
    rmsd_df.index.name = "frame"
    return rmsd_df

In [ ]:
# --

rmsd = rmsd_for_atomgroups(mda_universe, "backbone", ["protein"])
rmsd.head()

In [61]:
# --

import matplotlib.pyplot as plt

rmsd.plot(title="RMSD of protein")
plt.ylabel("RMSD (Å)"); #TODO: why is there a semicolon here
# this cell does not even have plt.show(); why is there a plot
# plot is of a data frame from numpy, but plt.ylabel is of matplotlib, why is the latter affecting the former

In [62]:
# --

from MDAnalysis.analysis import diffusionmap

def RMSD_dist_frames(universe, selection):
    """Calculate the RMSD between all frames in a matrix.

    Parameters
    ----------
    universe: MDAnalysis.core.universe.Universe
        MDAnalysis universe.
    selection: str
        Selection string for the atomgroup to be investigated, also used during alignment.

    Returns
    -------
    array: np.ndarray
        Numpy array of RMSD values.
    """
    pairwise_rmsd = diffusionmap.DistanceMatrix(universe, select=selection)
    pairwise_rmsd.run()
    return pairwise_rmsd.dist_matrix

In [ ]:
# --

dist_matrix_protein = RMSD_dist_frames(mda_universe, "protein")

In [ ]:
# get maximal distance between all frames
max_dist = np.amax(dist_matrix_protein)

In [ ]:
# --

fig, ax = plt.subplots(1,1)
fig.suptitle("RMSD between the frames")

# protein image
img1 = ax.imshow(dist_matrix_protein, cmap="viridis", vmin=0, vmax=max_dist)
ax.title.set_text("protein")
ax.set_xlabel("frames")
ax.set_ylabel("frames")


fig.colorbar(img1, ax=ax, orientation="horizontal", fraction=0.1, label="RMSD (Å)")

In [ ]:
# --

protein_residue_id1 = "53"
protein_residue_id2 = "24"
view = visualize_mda_universe(mda_universe, 
  sel_string = f"resid {protein_residue_id1} or (resid {protein_residue_id2})", \
    style = {'stick':{}})
view.show()

Peptide flip motion as distance

In [40]:
# --

protein_atom_name1 = "N"

protein_atom_name2 = "OE1"

In [41]:
# --

atomgroup_protein1 = mda_universe.select_atoms(
    f"resid {protein_residue_id1} and name {protein_atom_name1}")
atomgroup_protein2 = mda_universe.select_atoms(
    f"resid {protein_residue_id2} and name {protein_atom_name2}")

In [ ]:
# --

from MDAnalysis.analysis.distances import dist

distance = dist(atomgroup_protein1, atomgroup_protein2)
print(
    f"The distance between residues {protein_residue_id1} and {protein_residue_id2} is {distance[2][0]:.2f} Å"
)

In [43]:
# --

distances = []
for _ in mda_universe.trajectory:
    distance = dist(atomgroup_protein1, atomgroup_protein2)
    distances.append(distance[2][0])

In [ ]:
# --

plt.figure(figsize=(10, 6))
plt.plot(distances)
#plt.gca().set_ylim(bottom=0)
plt.ylabel("distance (Å)")
plt.xlabel("frame")
plt.title(f"Atomic distance between atoms of residues {protein_residue_id1} and {protein_residue_id2}")
plt.show()

In [ ]:
# --

min(distances)

In [ ]:
# --

distances[29]

The 30th frame is in the peptide flip "in" conformation

In [ ]:
# Select the specific frame you want to write
frame_index = 29  # For example, the 30th frame (index starts at 0)
mda_universe.trajectory[frame_index]  # Set the trajectory to the desired frame

# Select the atoms you want to write (e.g., the entire universe or a subset)
selected_atoms = mda_universe.select_atoms("all")  # Adjust the selection as needed

# Write the selected frame to a PDB file
# with mda.Writer("2K39_30.pdb", selected_atoms.n_atoms) as writer:
#     writer.write(selected_atoms)

print("Frame written to 2K39_30.pdb")

In [ ]:
from MDAnalysis.analysis import align

# Set the trajectory pointer to the desired reference frame
reference_frame = 29  # For example, the 30th frame (index starts at 0)
universe=mda_universe.trajectory[reference_frame]

DATA2 = "/home/uni/capstone/structures/cluster0/"
DATA3 = "/home/uni/capstone/structures/cluster1/"
DATA4 = "/home/uni/capstone/structures/cluster2/"
DATA5 = "/home/uni/capstone/structures/cluster3/"
DATA6 = "/home/uni/capstone/structures/cluster4/"
DATA7 = "/home/uni/capstone/structures/cluster5/"
DATA8 = "/home/uni/capstone/structures/cluster6/"
DATA9 = "/home/uni/capstone/structures/cluster7/"
DATA10 = "/home/uni/capstone/structures/cluster8/"
DATA11 = "/home/uni/capstone/structures/cluster9/"
DATA12 = "/home/uni/capstone/structures/cluster10/"
DATA13 = "/home/uni/capstone/structures/cluster11/"
DATA14 = "/home/uni/capstone/structures/cluster12/"
DATA15 = "/home/uni/capstone/structures/cluster13/"
DATA16 = "/home/uni/capstone/structures/cluster14/"
DATA17 = "/home/uni/capstone/structures/cluster15/"
DATA18 = "/home/uni/capstone/structures/cluster16/"
DATA19 = "/home/uni/capstone/structures/cluster17/"
DATA20 = "/home/uni/capstone/structures/cluster18/"
DATA21 = "/home/uni/capstone/structures/cluster19/"
print(DATA21)
sig30scale70c0v1 = mda.Universe(os.path.join(DATA2, "11_47_042.r0_min.pdb"))
sig30scale70c0v2 = mda.Universe(os.path.join(DATA2, "18_47_042.r24_min.pdb"))
sig30scale70c1v1 = mda.Universe(os.path.join(DATA3, "5_47_042.r20_min.pdb"))
sig30scale70c1v2 = mda.Universe(os.path.join(DATA3, "5_47_042.r64_min.pdb"))
sig30scale70c2v1 = mda.Universe(os.path.join(DATA4, "1_47_042.r36_min.pdb"))
sig30scale70c2v2 = mda.Universe(os.path.join(DATA4, "1_47_042.r44_min.pdb"))
sig30scale70c3v1 = mda.Universe(os.path.join(DATA5, "21_47_042.r32_min.pdb"))
sig30scale70c3v2 = mda.Universe(os.path.join(DATA5, "21_47_042.r49_min.pdb"))
sig30scale70c4v1 = mda.Universe(os.path.join(DATA6, "3_47_042.r29_min.pdb"))
sig30scale70c4v2 = mda.Universe(os.path.join(DATA6, "3_47_042.r51_min.pdb"))
sig30scale70c5v1 = mda.Universe(os.path.join(DATA7, "6_47_042.r24_min.pdb"))
sig30scale70c5v2 = mda.Universe(os.path.join(DATA7, "6_47_042.r54_min.pdb"))
sig30scale70c6v1 = mda.Universe(os.path.join(DATA8, "5_47_042.r4_min.pdb"))
sig30scale70c6v2 = mda.Universe(os.path.join(DATA8, "8_47_042.r40_min.pdb"))
sig30scale70c7v1 = mda.Universe(os.path.join(DATA9, "37_47_042.r12_min.pdb"))
sig30scale70c7v2 = mda.Universe(os.path.join(DATA9, "45_47_042.r24_min.pdb"))
sig30scale70c8v1 = mda.Universe(os.path.join(DATA10, "8_47_042.r29_min.pdb"))
sig30scale70c8v2 = mda.Universe(os.path.join(DATA10, "9_47_042.r32_min.pdb"))
sig30scale70c9v1 = mda.Universe(os.path.join(DATA11, "19_47_042.r59_min.pdb"))
sig30scale70c9v2 = mda.Universe(os.path.join(DATA11, "23_47_042.r21_min.pdb"))
sig30scale70c10v1 = mda.Universe(os.path.join(DATA12, "2_47_042.r8_min.pdb"))
sig30scale70c10v2 = mda.Universe(os.path.join(DATA12, "2_47_042.r58_min.pdb"))
sig30scale70c11v1 = mda.Universe(os.path.join(DATA13, "8_47_042.r18_min.pdb"))
sig30scale70c11v2 = mda.Universe(os.path.join(DATA13, "9_47_042.r26_min.pdb"))
sig30scale70c12v1 = mda.Universe(os.path.join(DATA14, "11_47_042.r17_min.pdb"))
sig30scale70c12v2 = mda.Universe(os.path.join(DATA14, "15_47_042.r59_min.pdb"))
sig30scale70c13v1 = mda.Universe(os.path.join(DATA15, "3_47_042.r0_min.pdb"))
#sig30scale70c13v2 = mda.Universe(os.path.join(DATA15, "3_47_042.r56_min.pdb"))
sig30scale70c14v1 = mda.Universe(os.path.join(DATA16, "10_47_042.r28_min.pdb"))
sig30scale70c14v2 = mda.Universe(os.path.join(DATA16, "10_47_042.r43_min.pdb"))
sig30scale70c15v1 = mda.Universe(os.path.join(DATA17, "4_47_042.r19_min.pdb"))
sig30scale70c15v2 = mda.Universe(os.path.join(DATA17, "4_47_042.r30_min.pdb"))
sig30scale70c16v1 = mda.Universe(os.path.join(DATA18, "16_47_042.r6_min.pdb"))
sig30scale70c16v2 = mda.Universe(os.path.join(DATA18, "25_47_042.r14_min.pdb"))
sig30scale70c17v1 = mda.Universe(os.path.join(DATA19, "2_47_042.r0_min.pdb"))
sig30scale70c17v2 = mda.Universe(os.path.join(DATA19, "2_47_042.r1_min.pdb"))
sig30scale70c18v1 = mda.Universe(os.path.join(DATA20, "3_47_042.r44_min.pdb"))
sig30scale70c18v2 = mda.Universe(os.path.join(DATA20, "3_47_042.r45_min.pdb"))
sig30scale70c19v1 = mda.Universe(os.path.join(DATA21, "19_47_042.r45_min.pdb"))
sig30scale70c19v2 = mda.Universe(os.path.join(DATA21, "21_47_042.r54_min.pdb"))


In [ ]:
view1 = visualize_mda_universe(sig30scale70c0v1)
view1.show()


In [ ]:

# Align the trajectory to the reference frame
alignment = align.AlignTraj(
    mobile=sig30scale70c0v1,  # The trajectory to align
    reference=mda_universe,  # The reference structure
    select="backbone",  # Atom selection for alignment
    in_memory=True  # Load trajectory into memory for modification
)
alignment.run()

# Save the aligned trajectory if needed
#sig30scale70c0v1.atoms.write("EROS30_sig30scale70.pdb")

In [ ]:
from MDAnalysis.analysis import rms

# Load the two protein structures
#reference = mda.Universe("reference.pdb")  # Reference structure
#mobile = mda.Universe("mobile.pdb")        # Structure to compare

# Select the atoms for RMSD calculation (e.g., backbone atoms)
selection = "backbone"

# Perform the RMSD calculation
rmsd_analysisc0v1 = rms.RMSD(sig30scale70c0v1, mda_universe, select=selection)
rmsd_analysisc0v2 = rms.RMSD(sig30scale70c0v2, mda_universe, select=selection)
rmsd_analysisc1v1 = rms.RMSD(sig30scale70c1v1, mda_universe, select=selection)
rmsd_analysisc1v2 = rms.RMSD(sig30scale70c1v2, mda_universe, select=selection)
rmsd_analysisc2v1 = rms.RMSD(sig30scale70c2v1, mda_universe, select=selection)
rmsd_analysisc2v2 = rms.RMSD(sig30scale70c2v2, mda_universe, select=selection)
rmsd_analysisc3v1 = rms.RMSD(sig30scale70c3v1, mda_universe, select=selection)
rmsd_analysisc3v2 = rms.RMSD(sig30scale70c3v2, mda_universe, select=selection)
rmsd_analysisc4v1 = rms.RMSD(sig30scale70c4v1, mda_universe, select=selection)
rmsd_analysisc4v2 = rms.RMSD(sig30scale70c4v2, mda_universe, select=selection)
rmsd_analysisc5v1 = rms.RMSD(sig30scale70c5v1, mda_universe, select=selection)
rmsd_analysisc5v2 = rms.RMSD(sig30scale70c5v2, mda_universe, select=selection)
rmsd_analysisc6v1 = rms.RMSD(sig30scale70c6v1, mda_universe, select=selection)
rmsd_analysisc6v2 = rms.RMSD(sig30scale70c6v2, mda_universe, select=selection)
rmsd_analysisc7v1 = rms.RMSD(sig30scale70c7v1, mda_universe, select=selection)
rmsd_analysisc7v2 = rms.RMSD(sig30scale70c7v2, mda_universe, select=selection)
rmsd_analysisc8v1 = rms.RMSD(sig30scale70c8v1, mda_universe, select=selection)
rmsd_analysisc8v2 = rms.RMSD(sig30scale70c8v2, mda_universe, select=selection)
rmsd_analysisc9v1 = rms.RMSD(sig30scale70c9v1, mda_universe, select=selection)
rmsd_analysisc9v2 = rms.RMSD(sig30scale70c9v2, mda_universe, select=selection)
rmsd_analysisc10v1 = rms.RMSD(sig30scale70c10v1, mda_universe, select=selection)
rmsd_analysisc10v2 = rms.RMSD(sig30scale70c10v2, mda_universe, select=selection)
rmsd_analysisc11v1 = rms.RMSD(sig30scale70c11v1, mda_universe, select=selection)
rmsd_analysisc11v2 = rms.RMSD(sig30scale70c11v2, mda_universe, select=selection)
rmsd_analysisc12v1 = rms.RMSD(sig30scale70c12v1, mda_universe, select=selection)
rmsd_analysisc12v2 = rms.RMSD(sig30scale70c12v2, mda_universe, select=selection)
rmsd_analysisc13v1 = rms.RMSD(sig30scale70c13v1, mda_universe, select=selection)
#rmsd_analysisc13v2 = rms.RMSD(sig30scale70c13v2, mda_universe, select=selection)
rmsd_analysisc14v1 = rms.RMSD(sig30scale70c14v1, mda_universe, select=selection)
rmsd_analysisc14v2 = rms.RMSD(sig30scale70c14v2, mda_universe, select=selection)
rmsd_analysisc15v1 = rms.RMSD(sig30scale70c15v1, mda_universe, select=selection)
rmsd_analysisc15v2 = rms.RMSD(sig30scale70c15v2, mda_universe, select=selection)
rmsd_analysisc16v1 = rms.RMSD(sig30scale70c16v1, mda_universe, select=selection)
rmsd_analysisc16v2 = rms.RMSD(sig30scale70c16v2, mda_universe, select=selection)
rmsd_analysisc17v1 = rms.RMSD(sig30scale70c17v1, mda_universe, select=selection)
rmsd_analysisc17v2 = rms.RMSD(sig30scale70c17v2, mda_universe, select=selection)
rmsd_analysisc18v1 = rms.RMSD(sig30scale70c18v1, mda_universe, select=selection)
rmsd_analysisc18v2 = rms.RMSD(sig30scale70c18v2, mda_universe, select=selection)
rmsd_analysisc19v1 = rms.RMSD(sig30scale70c19v1, mda_universe, select=selection)
rmsd_analysisc19v2 = rms.RMSD(sig30scale70c19v2, mda_universe, select=selection)
rmsd_analysisc0v1.run()
rmsd_analysisc0v2.run()
rmsd_analysisc1v1.run()
rmsd_analysisc1v2.run()
rmsd_analysisc2v1.run()
rmsd_analysisc2v2.run()
rmsd_analysisc3v1.run()
rmsd_analysisc3v2.run()
rmsd_analysisc4v1.run()
rmsd_analysisc4v2.run()
rmsd_analysisc5v1.run()
rmsd_analysisc5v2.run()
rmsd_analysisc6v1.run()
rmsd_analysisc6v2.run()
rmsd_analysisc7v1.run()
rmsd_analysisc7v2.run()
rmsd_analysisc8v1.run()
rmsd_analysisc8v2.run()
rmsd_analysisc9v1.run()
rmsd_analysisc9v2.run()
rmsd_analysisc10v1.run()
rmsd_analysisc10v2.run()
rmsd_analysisc11v1.run()
rmsd_analysisc11v2.run()
rmsd_analysisc12v1.run()
rmsd_analysisc12v2.run()
rmsd_analysisc13v1.run()
#rmsd_analysisc13v2.run()
rmsd_analysisc14v1.run()
rmsd_analysisc14v2.run()
rmsd_analysisc15v1.run()
rmsd_analysisc15v2.run()
rmsd_analysisc16v1.run()
rmsd_analysisc16v2.run()
rmsd_analysisc17v1.run()
rmsd_analysisc17v2.run()
rmsd_analysisc18v1.run()
rmsd_analysisc18v2.run()
rmsd_analysisc19v1.run()
rmsd_analysisc19v2.run()

# Get the RMSD value
rmsd_valuec0v1 = rmsd_analysisc0v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec0v2 = rmsd_analysisc0v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec1v1 = rmsd_analysisc1v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec1v2 = rmsd_analysisc1v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec2v1 = rmsd_analysisc2v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec2v2 = rmsd_analysisc2v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec3v1 = rmsd_analysisc3v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec3v2 = rmsd_analysisc3v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec4v1 = rmsd_analysisc4v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec4v2 = rmsd_analysisc4v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec5v1 = rmsd_analysisc5v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec5v2 = rmsd_analysisc5v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec6v1 = rmsd_analysisc6v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec6v2 = rmsd_analysisc6v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec7v1 = rmsd_analysisc7v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec7v2 = rmsd_analysisc7v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec8v1 = rmsd_analysisc8v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec8v2 = rmsd_analysisc8v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec9v1 = rmsd_analysisc9v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec9v2 = rmsd_analysisc9v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec10v1 = rmsd_analysisc10v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec10v2 = rmsd_analysisc10v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec11v1 = rmsd_analysisc11v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec11v2 = rmsd_analysisc11v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec12v1 = rmsd_analysisc12v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec12v2 = rmsd_analysisc12v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec13v1 = rmsd_analysisc13v1.rmsd[0, 2]  # RMSD value for the first frame
#rmsd_valuec13v2 = rmsd_analysisc13v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec14v1 = rmsd_analysisc14v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec14v2 = rmsd_analysisc14v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec15v1 = rmsd_analysisc15v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec15v2 = rmsd_analysisc15v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec16v1 = rmsd_analysisc16v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec16v2 = rmsd_analysisc16v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec17v1 = rmsd_analysisc17v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec17v2 = rmsd_analysisc17v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec18v1 = rmsd_analysisc18v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec18v2 = rmsd_analysisc18v2.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec19v1 = rmsd_analysisc19v1.rmsd[0, 2]  # RMSD value for the first frame
rmsd_valuec19v2 = rmsd_analysisc19v2.rmsd[0, 2]  # RMSD value for the first frame
print(f"c0v1 RMSD: {rmsd_valuec0v1:.2f} Å")
print(f"c0v2 RMSD: {rmsd_valuec0v2:.2f} Å")
print(f"c1v1 RMSD: {rmsd_valuec1v1:.2f} Å")
print(f"c1v2 RMSD: {rmsd_valuec1v2:.2f} Å")
print(f"c2v1 RMSD: {rmsd_valuec2v1:.2f} Å")
print(f"c2v2 RMSD: {rmsd_valuec2v2:.2f} Å")
print(f"c3v1 RMSD: {rmsd_valuec3v1:.2f} Å")
print(f"c3v2 RMSD: {rmsd_valuec3v2:.2f} Å")
print(f"c4v1 RMSD: {rmsd_valuec4v1:.2f} Å")
print(f"c4v2 RMSD: {rmsd_valuec4v2:.2f} Å")
print(f"c5v1 RMSD: {rmsd_valuec5v1:.2f} Å")
print(f"c5v2 RMSD: {rmsd_valuec5v2:.2f} Å")
print(f"c6v1 RMSD: {rmsd_valuec6v1:.2f} Å")
print(f"c6v2 RMSD: {rmsd_valuec6v2:.2f} Å")
print(f"c7v1 RMSD: {rmsd_valuec7v1:.2f} Å")
print(f"c7v2 RMSD: {rmsd_valuec7v2:.2f} Å")
print(f"c8v1 RMSD: {rmsd_valuec8v1:.2f} Å")
print(f"c8v2 RMSD: {rmsd_valuec8v2:.2f} Å")
print(f"c9v1 RMSD: {rmsd_valuec9v1:.2f} Å")
print(f"c9v2 RMSD: {rmsd_valuec9v2:.2f} Å")
print(f"c10v1 RMSD: {rmsd_valuec10v1:.2f} Å")
print(f"c10v2 RMSD: {rmsd_valuec10v2:.2f} Å")
print(f"c11v1 RMSD: {rmsd_valuec11v1:.2f} Å")
print(f"c11v2 RMSD: {rmsd_valuec11v2:.2f} Å")
print(f"c12v1 RMSD: {rmsd_valuec12v1:.2f} Å")
print(f"c12v2 RMSD: {rmsd_valuec12v2:.2f} Å")
print(f"c13v1 RMSD: {rmsd_valuec13v1:.2f} Å")
#print(f"c13v2 RMSD: {rmsd_valuec13v2:.2f} Å")
print(f"c14v1 RMSD: {rmsd_valuec14v1:.2f} Å")
print(f"c14v2 RMSD: {rmsd_valuec14v2:.2f} Å")
print(f"c15v1 RMSD: {rmsd_valuec15v1:.2f} Å")
print(f"c15v2 RMSD: {rmsd_valuec15v2:.2f} Å")
print(f"c16v1 RMSD: {rmsd_valuec16v1:.2f} Å")
print(f"c16v2 RMSD: {rmsd_valuec16v2:.2f} Å")
print(f"c17v1 RMSD: {rmsd_valuec17v1:.2f} Å")
print(f"c17v2 RMSD: {rmsd_valuec17v2:.2f} Å")
print(f"c18v1 RMSD: {rmsd_valuec18v1:.2f} Å")
print(f"c18v2 RMSD: {rmsd_valuec18v2:.2f} Å")
print(f"c19v1 RMSD: {rmsd_valuec19v1:.2f} Å")
print(f"c19v2 RMSD: {rmsd_valuec19v2:.2f} Å")

In [ ]:
# Align the trajectory to the reference frame
# alignment = align.AlignTraj(
#     mobile=sig30scale70c1v1,  # The trajectory to align
#     reference=mda_universe,  # The reference structure
#     select="backbone",  # Atom selection for alignment
#     in_memory=True  # Load trajectory into memory for modification
# )
# alignment.run()
# Perform the RMSD calculation
rmsd_analysis = rms.RMSD(sig30scale70c1v1, mda_universe, select=selection)
rmsd_analysis.run()

# Get the RMSD value
rmsd_value = rmsd_analysis.rmsd[0, 2]  # RMSD value for the first frame
print(f"RMSD: {rmsd_value:.2f} Å")